In [5]:
import transformers
from outlines import from_transformers, Generator
from pydantic import BaseModel
from typing import List, Literal

In [2]:
class Character(BaseModel):
    name: str
    age: int
    skills: List[str]

In [3]:
model = from_transformers(
    transformers.AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0"),
    transformers.AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0"),
)

In [4]:
generator = Generator(model, Character)
result = generator("Create a character")

# Call it with the output type to generate structured text
print(result) # '{"name": "Evelyn", "age": 34, "skills": ["archery", "stealth", "alchemy"]}'
#print(Character.model_validate_json(result))


W0930 11:28:37.938000 13276 site-packages\torch\_dynamo\convert_frame.py:1339] WON'T CONVERT _apply_token_bitmask_inplace_kernel c:\Users\runet\miniconda3\envs\outlining\Lib\site-packages\outlines_core\kernels\torch.py line 43 
W0930 11:28:37.938000 13276 site-packages\torch\_dynamo\convert_frame.py:1339] due to: 
W0930 11:28:37.938000 13276 site-packages\torch\_dynamo\convert_frame.py:1339] Traceback (most recent call last):
W0930 11:28:37.938000 13276 site-packages\torch\_dynamo\convert_frame.py:1339]   File "c:\Users\runet\miniconda3\envs\outlining\Lib\site-packages\torch\_dynamo\convert_frame.py", line 1272, in __call__
W0930 11:28:37.938000 13276 site-packages\torch\_dynamo\convert_frame.py:1339]     result = self._inner_convert(
W0930 11:28:37.938000 13276 site-packages\torch\_dynamo\convert_frame.py:1339]              ^^^^^^^^^^^^^^^^^^^^
W0930 11:28:37.938000 13276 site-packages\torch\_dynamo\convert_frame.py:1339]   File "c:\Users\runet\miniconda3\envs\outlining\Lib\site-packa

{"name": "Jane", "age":20, "skills": ["Cooking", "Dancing", "Art"] }


In [9]:
output_type = Literal["Negative", "Positive", "Neutral"]

In [15]:
class Semantic:
    verb: str
    object: str
    subject: str

In [16]:
generator = Generator(model, Semantic)
result = generator("Apply semantic role labelling to this sentence: The immigrants are causing the decline of our economy.")

TypeError: Type Semantic is currently not supported. Please open an issue: https://github.com/dottxt-ai/outlines/issues

In [ ]:
model = from_transformers(
    transformers.AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0"),
    transformers.AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0"),
)

In [57]:
import outlines
from outlines import Generator
from pydantic import BaseModel, Field
from typing import List

# Define the structured output schema for semantic role labeling
class Argument(BaseModel):
    role: str = Field(description="Semantic role (e.g., ARG0/Agent, ARG1/Patient, ARGM-TMP/Time, ARGM-LOC/Location)")
    text: str = Field(description="The text span filling this role")

class Predicate(BaseModel):
    predicate: str = Field(description="The main predicate/verb")
    agent: str = Field(description="ARG0: The entity performing the action (who/what does it)")
    patient: str = Field(description="ARG1: The entity affected by the action (who/what is affected)", default="")
    arguments: List[Argument] = Field(description="List of semantic arguments for this predicate")

class SemanticRoleLabeling(BaseModel):
    sentence: str = Field(description="The original sentence")
    predicates: List[Predicate] = Field(description="List of predicates with their semantic roles")

# Create a structured generator
generator = Generator(model, SemanticRoleLabeling)

# Example text for semantic role labeling
text = "John ate an apple in the park yesterday."

# Create the prompt
prompt = f"""Perform semantic role labeling on the following sentence. Identify all predicates and their semantic arguments.

Sentence: {text}

Common semantic roles:
- ARG0: Agent (who performs the action)
- ARG1: Patient/Theme (what is affected by the action)
- ARG2: Recipient, Beneficiary, or Instrument
- ARGM-TMP: Temporal (when)
- ARGM-LOC: Location (where)
- ARGM-MNR: Manner (how)
- ARGM-PRP: Purpose (why)

Analyze the sentence and output the semantic role labeling in the specified JSON format."""

# Generate structured output
result = generator(prompt, max_new_tokens=2048)

'''
# Print the results
print("Original sentence:", result.sentence)
print("\nSemantic Role Labeling:")
for pred in result.predicates:
    print(f"\nPredicate: {pred.predicate}")
    for arg in pred.arguments:
        print(f"  {arg.role}: {arg.text}")

'''

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


'\n# Print the results\nprint("Original sentence:", result.sentence)\nprint("\nSemantic Role Labeling:")\nfor pred in result.predicates:\n    print(f"\nPredicate: {pred.predicate}")\n    for arg in pred.arguments:\n        print(f"  {arg.role}: {arg.text}")\n\n'

In [96]:
# Define the structured output schema for semantic role labeling
class Blame(BaseModel):
    blame: bool = Field(description="Is blame present in the given sentence?")
    blamee: str = Field(description="The patient that is being blamed for something (Who / What is affected)")
    arguments: List[str] = Field(description="List of arguments that the blamee is being blamed for causing")
    text: str = Field(description="Part of the original text that identifies the blame")

#class BlamerBlamee(BaseModel):
    #blamer: str = Field(description="Blamer: The agent blaming another patient")
    #blamee: str = Field(description="Blamee: The patient that is being blamed for something. (Who / What is affected)")
    #arguments: List[Blame] = Field(description="List of arguments that the blamee is being blamed for causing")

class Blaming(BaseModel):
    sentence: str = Field(description="The original sentence")
    arguments: List[Blame] = Field(description="List of who or what is being blamed, and for what they are being blamed")

# Create a structured generator
generator = Generator(model, Blaming)

# Example text for semantic role labeling
text = "The immigrants are not causing the economic recession."

# Create the prompt
prompt = f"""Perform blame identification only on the following sentence, do not generate alternate sentences. 

Sentence: {text}

Rules:
- Start by looking into whether or not blame is present at all in the sentence 
- then identify who is being blamed, what they are being blamed for, and the arguments the person blaming them are using.
- Set blame=true ONLY if someone/something is being blamed for causing a negative outcome
- Set blame=false if the sentence explicitly denies blame (uses "not", "never", "no")
- The "text" field must be EXACTLY the sentence provided above - do not modify it
- Do NOT create alternative phrasings or opposite statements
- If the sentence says "X is NOT causing Y", this means NO blame (blame=false)

The relevant semantic roles:
- Blamee: The patient receiving the blame (who or what is being blamed for something)
- Argument: What is the blamee being blamed for.

Analyze the sentence and output your result in the specified JSON format."""

# Generate structured output
result = generator(prompt, max_new_tokens=512)



In [98]:
print(result)

{"sentence": "The immigrants are not causing the economic recession.","arguments": [{"blame": true,"blamee": "immigrants","arguments": [], "text": "The immigrants are not causing the economic recession."},{"blame": false,"blamee": "economy","arguments": [], "text": "The economy is causing the economic recession."},{"blame": false,"blamee": "economic recession","arguments": [], "text": "The economy is not causing the economic recession."},{"blame": false,"blamee": "immigrants","arguments": [], "text": "The immigrants are not causing the economic recession."},{"blame": false,"blamee": "economy","arguments": [], "text": "The economy is not causing the economic recession."},{"blame": false,"blamee": "immigrants","arguments": [], "text": "The immigrants are not causing the economic recession."},{"blame": false,"blamee": "economy","arguments": [], "text": "The economy is causing the economic recession."},{"blame": false,"blamee": "immigrants","arguments": [], "text": "The immigrants are not 

In [97]:
data = json.loads(result)
print(json.dumps(data, indent=2))

JSONDecodeError: Unterminated string starting at: line 1 column 1638 (char 1637)

In [82]:
result_out = Blaming.model_validate_json(result)

ValidationError: 2 validation errors for Blaming
predicates.0.arguments.0.blame_confidence
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than_equal
predicates.1.arguments.0.blame_confidence
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than_equal

In [78]:
import json
with open("srl_result_blame.json", "w") as f:
       json.dump(result_out.model_dump(), f, indent=2)

In [51]:
print(result_out)


sentence='John ate an apple in the park yesterday' predicates=[Predicate(predicate='ARGM', arguments=[Argument(role='ARGM_LOC', text='in'), Argument(role='ARGM_MNR', text='at'), Argument(role='ARGM_PRP', text='the'), Argument(role='ARGM_LOC', text='park'), Argument(role='ARGM_MNR', text='in'), Argument(role='ARGM_PRP', text='the'), Argument(role='ARGM_LOC', text='last'), Argument(role='ARGM_MNR', text='night')])]


In [25]:
print(result_out)

sentence='John ate an apple in the park yesterday' predicates=[Predicate(predicate='ARGM', arguments=[Argument(role='ARGM_LOC', text='in'), Argument(role='ARGM_MNR', text='at'), Argument(role='ARGM_PRP', text='the'), Argument(role='ARGM_LOC', text='park'), Argument(role='ARGM_MNR', text='in'), Argument(role='ARGM_PRP', text='the'), Argument(role='ARGM_LOC', text='last'), Argument(role='ARGM_MNR', text='night')])]


In [44]:
# Example with a more complex sentence
text2 = "Mary gave her friend a book at the library on Monday because she wanted to help."

prompt2 = f"""Perform semantic role labeling on the following sentence. Identify all predicates and their semantic arguments.

Sentence: {text2}

Common semantic roles in order of relevance:
- ARG0: Agent (who performs the action)
- ARG1: Patient/Theme (what is affected by the action)
- ARG2: Recipient, Beneficiary, or Instrument
- ARGM-TMP: Temporal (when)
- ARGM-LOC: Location (where)
- ARGM-MNR: Manner (how)
- ARGM-PRP: Purpose (why)

Analyze the sentence and output the semantic role labeling in the specified JSON format, focusing on ARG0, ARG1, and ARG2 roles."""

result2 = generator(prompt2,max_new_tokens=512)

'''
print("\n" + "="*50)
print("\nOriginal sentence:", result2.sentence)
print("\nSemantic Role Labeling:")
for pred in result2.predicates:
    print(f"\nPredicate: {pred.predicate}")
    for arg in pred.arguments:
        print(f"  {arg.role}: {arg.text}")
'''

'\nprint("\n" + "="*50)\nprint("\nOriginal sentence:", result2.sentence)\nprint("\nSemantic Role Labeling:")\nfor pred in result2.predicates:\n    print(f"\nPredicate: {pred.predicate}")\n    for arg in pred.arguments:\n        print(f"  {arg.role}: {arg.text}")\n'

In [46]:
result2_out = SemanticRoleLabeling.model_validate_json(result2)

In [45]:
print(result2)

{"sentence": "Mary gave her friend a book at the library on Monday because she wanted to help.", "predicates": [{"predicate": "gave", "arguments": [{"role": "ARGM_LOC", "text": "at"}, {"role": "ARGM_MNR", "text": "the library"}, {"role": "ARGM_PRP", "text": "on"}, {"role": "ARGM_LOC", "text": "Monday"}, {"role": "ARGM_MNR", "text": "because"}, {"role": "ARGM_PRP", "text": "she"}, {"role": "ARGM_LOC", "text": "helped"}]}]}


In [62]:
import json
with open("srl_result_blame.json", "w") as f:
       json.dump(result_out.model_dump(), f, indent=2)

In [47]:
import json
with open("srl_result2.json", "w") as f:
       json.dump(result2_out.model_dump(), f, indent=2)